In [5]:
import socket
import threading


In [7]:
class ChatServer:
    def __init__(self, host='localhost', port=12345):
        self.server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.server.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        self.server.bind((host, port))
        self.server.listen(5)
        print(f"Server started on {host}:{port}")
        self.clients = {}
        self.usernames = {}
        self.message_history = []

    def broadcast(self, message, sender_socket=None, exclude_sender=False):
        for client, username in self.clients.items():
            if exclude_sender and client == sender_socket:
                continue
            try:
                client.send(message)
            except:
                self.remove_client(client)

    def send_private_message(self, message, recipient_username):
        recipient_socket = None
        for client, username in self.clients.items():
            if username == recipient_username:
                recipient_socket = client
                break
        if recipient_socket:
            try:
                recipient_socket.send(message)
            except:
                self.remove_client(recipient_socket)

    def handle_client(self, client_socket):
        username = client_socket.recv(1024).decode('utf-8')
        self.clients[client_socket] = username
        self.usernames[username] = client_socket
        self.broadcast(f"{username} has joined the chat.".encode('utf-8'), exclude_sender=True)
        
        for msg in self.message_history:
            client_socket.send(msg)

        while True:
            try:
                message = client_socket.recv(1024)
                if message:
                    decoded_message = message.decode('utf-8')
                    if decoded_message.startswith('/pm'):
                        _, recipient, pm_message = decoded_message.split(' ', 2)
                        self.send_private_message(f"[PM from {username}] {pm_message}".encode('utf-8'), recipient)
                    else:
                        formatted_message = f"{username}: {decoded_message}".encode('utf-8')
                        self.message_history.append(formatted_message)
                        if len(self.message_history) > 10:
                            self.message_history.pop(0)
                        self.broadcast(formatted_message)
            except:
                self.remove_client(client_socket)
                break

    def remove_client(self, client_socket):
        if client_socket in self.clients:
            username = self.clients[client_socket]
            del self.clients[client_socket]
            del self.usernames[username]
            self.broadcast(f"{username} has left the chat.".encode('utf-8'))
            client_socket.close()

    def run(self):
        while True:
            client_socket, client_address = self.server.accept()
            print(f"New connection: {client_address}")
            threading.Thread(target=self.handle_client, args=(client_socket,)).start()

# Start the server
server = ChatServer()
threading.Thread(target=server.run).start()

Server started on localhost:12345
New connection: ('127.0.0.1', 60695)
New connection: ('127.0.0.1', 60748)
New connection: ('127.0.0.1', 60860)
New connection: ('127.0.0.1', 60910)
